In [1]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import transformers
import pandas as pd
import datasets
import re

modelpath = "./../models/checkpoint-4455/"

tokenizer = transformers.AutoTokenizer.from_pretrained(
    modelpath,

    model_max_length=100,
    padding_side="right",
    use_fast=True,
    trust_remote_code=True,
)

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    modelpath,
    cache_dir=None,
    num_labels=1,
    trust_remote_code=True,
)

def tokenize_function(example):
        return tokenizer(example["seq"], truncation=True)

def post():
        dna = "Atggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgcttcagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagcacccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagtaa"
        dna = dna.upper()




        indexs = [m.start() for m in re.finditer( '(?=GG)', dna ) if m.start() > 20]
        gRNA = []
        Cut_Pos = []
        PAM = []
        
        for i in indexs:
            gRNA.append( dna[i - 21:i] )
            Cut_Pos.append( i - 4 )
            PAM.append( dna[i - 1:i + 2] )

    # print("Before reverse")
    # print("Strand ",  Strand)
    # print("gRNAS Shape", len(gRNA))
    # print("gRNA ", gRNA)
    # print("Cut_Pos ", Cut_Pos)
    # print("PAM ", PAM)
    # print("\n")


    # sequence_complement = str( Seq.Seq( sequence ).reverse_complement() )

    # index_reverse = [m.start() for m in re.finditer( '(?=GG)', sequence_complement ) if m.start() > 20]

    # for i in index_reverse:
    #     Strand.append( '-' )
    #     gRNA.append( sequence_complement[i - 21:i] )
    #     Cut_Pos.append( i - 4 )
    #     PAM.append( sequence_complement[i - 1:i + 2] )




        gRNA = pd.DataFrame({'seq' : gRNA})
        raw_dataset = datasets.Dataset.from_pandas(gRNA)

        tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
        tokenized_dataset.set_format("torch")

        y_preds = []
        
        for row in tokenized_dataset:
            y_preds.append([row['seq'] ,model(row["input_ids"].unsqueeze(0), row["attention_mask"].unsqueeze(0))[0].item()])
            

        df_y_preds = pd.DataFrame(y_preds, columns=["seq", "Prediction"])
        df_y_preds.sort_values(by=['Prediction'], inplace=True, ascending=False)
        print(df_y_preds)



        

print(post())

c:\Users\robin\anaconda3\envs\dnabert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'transformers_modules.'